In [1]:
from Pipeline import Helper, Purchase_Probabilities
import pandas as pd

In [19]:
DATA_PATH   =   r'C:\Users\Benedikt\Documents\GitHub\mlim-g2\data'

EXPORT_PATH = r'C:\Users\Benedikt\Documents\GitHub\mlim-g2\data\export'

files = {
    'data': [
        #DATA_PATH + 'baskets.parquet',
        #DATA_PATH + 'coupon_index.parquet', 
        #DATA_PATH + 'coupons.parquet', 
        #DATA_PATH + '\merged.parquet',
        DATA_PATH + '\clean.parquet', 
        #DATA_PATH + 'prepare.parquet'
    ]}

In [27]:
eins = (1, 0)
eins[0]

1

In [20]:
class Elasticities(Helper):
    
    """calculate price elasticities
        input: basket df with variables 'product'(int), 'discount'(int/float), 'price'(int/float) (nan substituted with zero)
        output: basket df with 'elasticity' added for every product   
    """
    
    def __init__(self):
        super().__init__() 
        self.elast = None
    
    
    
    def calc_week_elastics(self, week, shopper):
        elast = []
        week_basket_count = (abs(shopper[0] - shopper[1]) + 1) * abs(week[0] - week[1])
        
        
        output = basket[(week[1] >= basket['week']) & (basket['week'] >= week[0])]      ##full input or decreased size?
       

        for i in range(250):
            reg_price = output[output['product'] == i]
            reg_price_buy = len(reg_price[reg_price['discount'] == 0])
            all_discounts_offers = len(reg_price[reg_price['discount'] > 0])
            reg_price_offer = total_basket_count - all_discounts_offers
            reg_price_buy_rate = reg_price_buy / reg_price_offer
            discount_30 = reg_price[reg_price['discount'] == 30]          ##check if discount is listed as 30 or 0.3 
            discount_30_offer = len(discount_30)
            discount_30_buy = (discount_30['price'] != 0).sum()
            discount_30_buy_rate = discount_30_buy / discount_30_offer
            elast.append((discount_30_buy_rate - reg_price_buy_rate) / (0.3 * reg_price_buy_rate))

        elast = pd.DataFrame(elast)
        elast['product'] = elast.index
        output = basket_temp.merge(elast, how = 'left')
        output = output.rename(columns = {0 : 'elasticity'})

        return output
    
    
    
    
    
    
    
class Redemptions(Helper):
    
    
    """calculate product and costumer specific coupon redemption likelihoods
        input: basket df with variables 'discount'(int/float), 'price'(int/float), 'product'(int), 'shopper'(int)
        output: basket df with 'product_redemption_likelihood', 'shopper_product_discount_usage', 'discount_buy' added
    
    """
    
    
    def __init__(self):
        super().__init__()
        
    def calc_coupon_rates(self,  week):
        
        basket_temp = basket[(week[1] >= basket['week']) & (basket['week'] >= week[0])]

        # coupon redemption likelihood by product
        coupons = basket_temp[basket_temp['discount'] > 0].copy()
        coupons.loc[(coupons['price'] != 0), 'redeemed']  = 1
        coupons['redeemed'] = coupons['redeemed'].fillna(0)

        redemption_rate = coupons.groupby(['product'])['redeemed'].mean()
        basket_temp = basket_temp.merge(redemption_rate, how = 'left', on = 'product')
        basket_temp = basket_temp.rename(columns = {'redeemed' : 'product_redemption_likelihood'})


        # coupon redemption likelihood by costumer & product
        costumer_redemption_rate = coupons.groupby(['shopper', 'product'])['redeemed'].mean()
        basket_temp = basket_temp.merge(costumer_redemption_rate, how = 'left', on = ['shopper', 'product'])
        basket_temp = basket_temp.rename(columns = {'redeemed' : 'shopper_product_discount_usage'})


        # only buy because of discount?
        buy_all = basket_temp.groupby(['shopper', 'product']).size()
        discount = coupons.groupby(['shopper', 'product']).size()
        discount_buy = discount / buy_all
        discount_buy = discount_buy.fillna(0)
        discount_buy = discount_buy.rename('discount_buy')
        basket_temp = basket_temp.merge(discount_buy, how = 'left', on = ['shopper', 'product'])

        return basket_temp

In [21]:
elastics = Purchase_Probabilities()

In [22]:
elastics.load(files)

In [24]:
elastics.describe()

AttributeError: 'Purchase_Probabilities' object has no attribute 'describe'

In [23]:
elastics.get_merged()

KeyError: 'coupons'

In [18]:
elastics.prepare(df='clean.parquet', shopper=(0,100), week=(0,10), product=(0,249), price_aggregation_fn='mode')


KeyError: '/clean.parquet'